In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error, r2_score
import os
from dotenv import load_dotenv
import yfinance as yf
import pandas as pd
import pandas_ta as ta
from scikeras.wrappers import KerasRegressor
# Load environment variables from the .env file
load_dotenv('../.env');

# Data Viz. 
import statsmodels.formula.api as smf
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.ndimage import gaussian_filter
from calendar import monthrange
from calendar import month_name

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix
import requests
import csv
from itertools import permutations
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.tsa.stattools import adfuller,kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_pacf

import statsmodels.graphics.tsaplots as tsaplot
from statsmodels.tsa.holtwinters import Holt, ExponentialSmoothing, SimpleExpSmoothing

import warnings
warnings.filterwarnings('ignore')

In [2]:
def load_and_prepare(ticker, start_date, end_date):
    df = pd.DataFrame().ta.ticker(ticker, start=start_date , end=end_date)
    #df.ta.adjusted = "adj_close"
    df.reset_index(inplace=True)
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    
    df['rsi']  = ta.rsi(df.close, length=15)
    df['emaf'] = ta.ema(df.close, length=20) #fast moving avg
    df['emam'] = ta.ema(df.close, length=100) #medium moving avg
    df['emas'] = ta.ema(df.close, length=150) #slow moving avg
    #df['macd'] = ta.macd(df.close, fast=12, slow=26, signal=9) #moving average convergance and divergance
    
    df['vma']  = ta.sma(df.volume, length=20) # Volume Moving Average
    df['vrc']  = ta.roc(df.volume, length=5) # Volume Rate of change
    df.drop(['dividends', 'stock_splits'], axis=1, inplace=True)
    #df['close'] = np.log(df['close'])
    #df['open'] = np.log(df['open'])
    df['target']            = ((df.close - df.open)/df.open)*100
    df['target']            = df.target.shift(-1)
    #df['target_class']      = [2 if df.target[i]>1 and 0 if df.target[i]< (-1) else 1 for i in range(len(df))]
    df['target_class'] = np.select([(df['target'] > 1), (df['target'] < -1), ((df['target'] >= -1) & (df['target'] <= 1))], [2, 0, 1], default=np.nan)
    df['target_next_close'] = df.close.shift(-1)
    df.dropna(inplace=True)
    return df

In [3]:
df = load_and_prepare("AMZN", "2018-03-20" , "2020-07-16")

In [4]:
amazon_sent = pd.read_csv('amazon_df.csv')

In [5]:
amazon_sent

,Headlines,Time,Description,Sentiment,Probabilities
0,Amazon India demands product listings have cou...,Jul 16 2020,Amazon.com Inc's India unit has told sellers t...,positive,[[0.02140732 0.02336918 0.95522356]]
1,India's Reliance seen emerging as bigger threa...,Jul 16 2020,A $20 billion fund raising spree may take Indi...,neutral,[[0.18353687 0.77087814 0.04558501]]
2,"Amazon extends work from home policy, still re...",Jul 15 2020,Amazon.com Inc has extended a work-from-home ...,positive,[[0.09093143 0.0315794 0.8774892 ]]
3,Amazon tests primary healthcare program for em...,Jul 14 2020,Amazon.com Inc said on Tuesday it launched a ...,positive,[[0.11197656 0.00946167 0.8785618 ]]
4,"Amazon.com bans, then un-bans TikTok app from ...",Jul 10 2020,"In the span of a few hours on Friday, Amazon.c...",neutral,[[0.01619002 0.5858641 0.3979459 ]]
...,...,...,...,...,...
518,Amazon in talks with Casino over Brazil electr...,Mar 27 2018,Amazon.com Inc and Casino Guichard Perrachon ...,positive,[[0.04936735 0.02370401 0.92692864]]
519,"Amazon, Casino in talks on possible Brazil par...",Mar 27 2018,Amazon.com Inc and French retail group Casino...,positive,[[0.06105959 0.01682694 0.9221134 ]]
520,Short visits rise at Whole Foods stores with A...,Mar 26 2018,Shoppers are making more quick trips to Whole ...,neutral,[[0.30819288 0.5831501 0.10865699]]
521,Amazon is now second most valuable U.S.-listed...,Mar 20 2018,Amazon.com became the second most valuable pu...,negative,[[0.666755 0.01781601 0.31542894]]


In [6]:
df

,date,open,high,low,close,volume,rsi,emaf,emam,emas,vma,vrc,target,target_class,target_next_close
149,2018-10-19 00:00:00-04:00,89.258003,90.455002,87.650002,88.201500,118144000,38.211428,92.473713,90.377979,86.630956,125582700.0,-37.454207,0.297085,1.0,89.464996
150,2018-10-22 00:00:00-04:00,89.199997,90.474998,87.800003,89.464996,90000000,41.564155,92.187169,90.359900,86.668493,125869000.0,-30.093830,1.518732,2.0,88.434998
151,2018-10-23 00:00:00-04:00,87.112000,88.817001,85.699997,88.434998,134478000,39.683430,91.829819,90.321783,86.691891,128054500.0,14.744279,-6.173534,0.0,83.209999
152,2018-10-24 00:00:00-04:00,88.684998,88.885498,82.828003,83.209999,138568000,31.850342,91.008884,90.180956,86.645773,130669400.0,30.843028,4.627964,2.0,89.108498
153,2018-10-25 00:00:00-04:00,85.167000,89.740501,84.600502,89.108498,205714000,44.985114,90.827895,90.159719,86.678392,136625700.0,75.105550,-0.411007,1.0,82.140503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,2020-07-08 00:00:00-04:00,151.130493,154.198502,150.621506,154.055496,100752000,76.319699,138.812875,118.831638,112.772499,94419200.0,33.633976,2.138647,2.0,159.131500
580,2020-07-09 00:00:00-04:00,155.799500,159.694000,153.699997,159.131500,127774000,79.261115,140.747982,119.629655,113.386525,95861900.0,0.397586,0.258166,1.0,160.000000
581,2020-07-10 00:00:00-04:00,159.587997,160.750000,156.785004,160.000000,109720000,79.722844,142.581508,120.429068,114.003922,95547800.0,-16.795583,-4.523446,0.0,155.199997
582,2020-07-13 00:00:00-04:00,162.552994,167.214493,153.419495,155.199997,154408000,70.436692,143.783268,121.117601,114.549566,97832100.0,12.205331,-0.161865,1.0,154.199997


In [13]:
# Assuming df is your stock dataframe and news_df is your news dataframe
df['date'] = pd.to_datetime(df['date'], utc=True)  # Convert to UTC timezone
amazon_sent['date'] = pd.to_datetime(amazon_sent['Time'], utc=True)

# Now, perform the merge using the 'date' column
merged_df = pd.merge(df, amazon_sent, on='date', how='left')

# Display the resulting merged DataFrame
merged_df

,date,open,high,low,close,volume,rsi,emaf,emam,emas,vma,vrc,target,target_class,target_next_close,Headlines,Time,Description,Sentiment,Probabilities
0,2018-10-19 04:00:00+00:00,89.258003,90.455002,87.650002,88.201500,118144000,38.211428,92.473713,90.377979,86.630956,125582700.0,-37.454207,0.297085,1.0,89.464996,NaN,NaT,NaN,NaN,NaN
1,2018-10-22 04:00:00+00:00,89.199997,90.474998,87.800003,89.464996,90000000,41.564155,92.187169,90.359900,86.668493,125869000.0,-30.093830,1.518732,2.0,88.434998,NaN,NaT,NaN,NaN,NaN
2,2018-10-23 04:00:00+00:00,87.112000,88.817001,85.699997,88.434998,134478000,39.683430,91.829819,90.321783,86.691891,128054500.0,14.744279,-6.173534,0.0,83.209999,NaN,NaT,NaN,NaN,NaN
3,2018-10-24 04:00:00+00:00,88.684998,88.885498,82.828003,83.209999,138568000,31.850342,91.008884,90.180956,86.645773,130669400.0,30.843028,4.627964,2.0,89.108498,NaN,NaT,NaN,NaN,NaN
4,2018-10-25 04:00:00+00:00,85.167000,89.740501,84.600502,89.108498,205714000,44.985114,90.827895,90.159719,86.678392,136625700.0,75.105550,-0.411007,1.0,82.140503,NaN,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,2020-07-08 04:00:00+00:00,151.130493,154.198502,150.621506,154.055496,100752000,76.319699,138.812875,118.831638,112.772499,94419200.0,33.633976,2.138647,2.0,159.131500,NaN,NaT,NaN,NaN,NaN
431,2020-07-09 04:00:00+00:00,155.799500,159.694000,153.699997,159.131500,127774000,79.261115,140.747982,119.629655,113.386525,95861900.0,0.397586,0.258166,1.0,160.000000,NaN,NaT,NaN,NaN,NaN
432,2020-07-10 04:00:00+00:00,159.587997,160.750000,156.785004,160.000000,109720000,79.722844,142.581508,120.429068,114.003922,95547800.0,-16.795583,-4.523446,0.0,155.199997,NaN,NaT,NaN,NaN,NaN
433,2020-07-13 04:00:00+00:00,162.552994,167.214493,153.419495,155.199997,154408000,70.436692,143.783268,121.117601,114.549566,97832100.0,12.205331,-0.161865,1.0,154.199997,NaN,NaT,NaN,NaN,NaN


In [14]:
merged_df.isna().sum()

date                   0
open                   0
high                   0
low                    0
close                  0
volume                 0
rsi                    0
emaf                   0
emam                   0
emas                   0
vma                    0
vrc                    0
target                 0
target_class           0
target_next_close      0
Headlines            435
Time                 435
Description          435
Sentiment            435
Probabilities        435
dtype: int64

In [25]:
merged_df.head(10)

,date,open,high,low,close,volume,rsi,emaf,emam,emas,vma,vrc,target,target_class,target_next_close,Headlines,Time,Description,Sentiment,Probabilities
0,2018-10-19 04:00:00+00:00,89.258003,90.455002,87.650002,88.201500,118144000,38.211428,92.473713,90.377979,86.630956,125582700.0,-37.454207,0.297085,1.0,89.464996,NaN,NaT,NaN,NaN,NaN
1,2018-10-22 04:00:00+00:00,89.199997,90.474998,87.800003,89.464996,90000000,41.564155,92.187169,90.359900,86.668493,125869000.0,-30.093830,1.518732,2.0,88.434998,NaN,NaT,NaN,NaN,NaN
2,2018-10-23 04:00:00+00:00,87.112000,88.817001,85.699997,88.434998,134478000,39.683430,91.829819,90.321783,86.691891,128054500.0,14.744279,-6.173534,0.0,83.209999,NaN,NaT,NaN,NaN,NaN
3,2018-10-24 04:00:00+00:00,88.684998,88.885498,82.828003,83.209999,138568000,31.850342,91.008884,90.180956,86.645773,130669400.0,30.843028,4.627964,2.0,89.108498,NaN,NaT,NaN,NaN,NaN
4,2018-10-25 04:00:00+00:00,85.167000,89.740501,84.600502,89.108498,205714000,44.985114,90.827895,90.159719,86.678392,136625700.0,75.105550,-0.411007,1.0,82.140503,NaN,NaT,NaN,NaN,NaN
5,2018-10-26 04:00:00+00:00,82.479500,84.922997,80.150002,82.140503,299276000,36.163323,90.000524,90.000923,86.618287,147504400.0,153.314599,-7.296385,0.0,76.944000,NaN,NaT,NaN,NaN,NaN
6,2018-10-29 04:00:00+00:00,83.000000,83.287003,74.750000,76.944000,277322000,31.264362,88.757046,89.742370,86.490151,157910000.0,208.135556,2.978152,2.0,76.521004,NaN,NaT,NaN,NaN,NaN
7,2018-10-30 04:00:00+00:00,74.307999,77.049500,73.818001,76.521004,249202000,30.899294,87.591708,89.480561,86.358109,164969400.0,85.310608,1.784726,2.0,79.900497,NaN,NaT,NaN,NaN,NaN
8,2018-10-31 04:00:00+00:00,78.499496,81.195503,78.254501,79.900497,187804000,37.178569,86.859212,89.290857,86.272578,169106500.0,35.532013,2.586954,2.0,83.276497,NaN,NaT,NaN,NaN,NaN
9,2018-11-01 04:00:00+00:00,81.176498,83.522499,79.921997,83.276497,162710000,42.747074,86.518001,89.171760,86.232895,169985000.0,-20.904751,-0.778034,1.0,83.276497,NaN,NaT,NaN,NaN,NaN


In [23]:
from scipy.interpolate import interp1d


# Identify NaN values and their indices
nan_indices = df.index[merged_df['Sentiment'].isna()].tolist()

# Function to fill NaN values using polynomial interpolation
def fill_nans_with_polynomial(series):
    # Non-NaN values
    non_nan_indices = series.index[~series.isna()]
    non_nan_values = series.dropna().values

    # Polynomial interpolation of degree 3
    poly_interpolation = interp1d(non_nan_indices, non_nan_values, kind='quadratic', fill_value='extrapolate')

    # Replace NaN values with interpolated values
    series.loc[nan_indices] = poly_interpolation(nan_indices)
    return series

# Apply the function to fill NaN values in the 'sentiment' column
merged_df['Sentiment'] = fill_nans_with_polynomial(merged_df['Sentiment'])

merged_df

ValueError: cannot reshape array of size 0 into shape (0,newaxis)